# Homework 2: Network Optimization and Non-linear Models

**Done by Javier Alzuaz & Jaime Lobato**

## 1. Part A: Network Optimization (Discrete Model)

**Topic:** **Optimization of the BiciMad Network Capacity (Optimal Slot Assignment)**

The objective is to minimize the total network cost, which includes the cost of capacity expansion and the penalty cost for user rejection (when there is no bike or no free dock), making the capacity ($\mathbf{S_i}$) a discrete variable. [Image of a network optimization model diagram showing facilities and customer zones with flow and capacity constraints]

### Parameters (Input Data)

| Parameter | Description | Source Value |
| :--- | :--- | :--- |
| $N$ | Set of all BiciMad stations. | (Network nodes, over 600). |
| $D_{ij}$ | Total number of trips from station $i$ to station $j$. | `matriz_demanda_Dij.csv` |
| $c_{ij}$ | Unit cost of movement/transport from $i$ to $j$. | `matriz_costos_Cij.csv` |
| $C^{\text{initial}}_i$ | Initial capacity of slots at station $i$. | `bikestationbicimad_csv.csv` |
| $C^{\text{slot}}$ | Cost to install a new dock/slot. | **Assumed**, e.g., $\mathbf{C^{\text{slot}} = €500}$ per slot. |
| $P^{\text{rejection}}$ | Penalty for user rejection. | **Assumed**, e.g., $\mathbf{P^{\text{rejection}} = €5}$ per rejection. |
| $B$ | Maximum budget for capacity expansion. | **Assumed**, e.g., $\mathbf{B = €100,000}$. |

### Decision Variables

* $\mathbf{S_i \in \mathbb{Z}^+}$: Final number of docks (slots) assigned to station $i$. (**Discrete/Integer** Variable).

### Mathematical Formulation

**Objective (Cost Minimization):**

Minimize the Capacity Expansion Cost plus the Rejection Penalty.
$$\min \sum_{i \in N} C^{\text{slot}} \cdot \max(0, S_i - C^{\text{initial}}_i) + \sum_{i \in N} P^{\text{rejection}} \cdot (\text{Outbound Rejection}_i + \text{Inbound Rejection}_i)$$

**Key Constraints (Subject to):**

1.  **Capacity Limit (Variable Domain):** The number of slots must be within a realistic operational range.
    $$\mathbf{S_{min} \leq S_i \leq S_{max}}, \quad \forall i \in N \quad \text{(e.g., } 10 \leq S_i \leq 50 \text{)}$$
2.  **Budget (If considered as a constraint):**
    $$\sum_{i \in N} C^{\text{slot}} \cdot \max(0, S_i - C^{\text{initial}}_i) \leq B$$
3.  **Loss Constraints (e.g., Inbound Rejection):** This constraint links the final capacity to the demand for arrival slots.
    $$\text{Inbound Rejection}_i = \max(0, \sum_{j} D_{ji} - S_i)$$

---


In [1]:
import pandas as pd
from pulp import *
import numpy as np

# We first load the data and define the parameters for the optimization model
# Demand matrix Dij: number of trips from station i to station j
demand_matrix_Dij = pd.read_csv('matriz_demanda_Dij.csv', index_col=0)

# Nodes data: information about each bike station
df_stations = pd.read_csv('bikestationbicimad_csv.csv', delimiter=';')
df_stations['number'] = pd.to_numeric(df_stations['number'], errors='coerce')
df_stations = df_stations.dropna(subset=['number'])
df_stations['ID_Estacion'] = df_stations['number'].astype(int)
df_initial_capacity = df_stations[['ID_Estacion', 'TotalBases']].set_index('ID_Estacion')

# Set of all stations
N = demand_matrix_Dij.index.tolist()  

# Initial capacity of each station
C_initial = df_initial_capacity['TotalBases'].to_dict()

# Assumed parameters
C_slot = 500         # Cost per installing a new slot (€)
P_rejection_out = 5.0  # Penalization for rejecting an outgoing trip (No slot available) (€/trip)
P_rejection_in = 5.0   # Penalization for rejecting an incoming trip (No slot available) (€/trip)
B_max = 100000       # Maximum budget for expansion (€)
S_min = 10           # Minimum capacity of slots per station
S_max = 50           # Maximum capacity of slots per station

In [2]:
# We compute the total demand for each station

# Total demand of outgoing trips from each station i 
D_out = demand_matrix_Dij.sum(axis=1).to_dict()

# Total demand of incoming trips to each station i
D_in = demand_matrix_Dij.sum(axis=0).to_dict()

# We define the optimization model and the decision variables

prob = LpProblem("BiciMad_Capacity_Optimization", LpMinimize)

# S_i: total number of slots at station i after expansion (discrete variable)
S = LpVariable.dicts("Final_Slots", N, lowBound=S_min, upBound=S_max, cat=LpInteger)

# We model the rejection variables

# R_in_i: number of trips that are rejected due to lack of slots for incoming bikes at station i.
R_in = LpVariable.dicts("Rejection_Inbound", N, lowBound=0, cat=LpContinuous)

# R_out_i: number of trips that are rejected due to lack of bikes in the outgoing trips from station i.
R_out = LpVariable.dicts("Rejection_Outbound", N, lowBound=0, cat=LpContinuous)

# E_i: capacity expansion at station i )
E = LpVariable.dicts("Capacity_Expansion", N, lowBound=0, cat=LpContinuous)

In [3]:
# Objective Function
# Min C_slot * SUM(Expansion) + P_rejection * SUM(Rejection)

prob += (
    # Cost 1: Cost of Capacity Expansion
    lpSum([C_slot * E[i] for i in N]) + 
    
    # Cost 2: Penalization for Rejection of Inbound Trips (No slot available)
    lpSum([P_rejection_in * R_in[i] for i in N]) +
    
    # Cost 3: Penalization for Rejection of Outbound Trips (No bike available)
    lpSum([P_rejection_out * R_out[i] for i in N])
)


# Key Constraints

# Constraint 1: Definition of the Expansion (Only counts if S_i > C_initial_i)
for i in N:
    prob += E[i] >= S[i] - C_initial.get(i, 0), f"Expansion_Def_{i}"

# Constraint 2: Definition of the Rejection of Inbound Trips (No slot available)
# The rejection is the excess of inbound demand over the final capacity S_i
for i in N:
    prob += R_in[i] >= D_in.get(i, 0) - S[i], f"Rejection_Inbound_Def_{i}"

# Constraint 3: Definition of the Rejection of Outbound Trips (No bike available)
for i in N:
    prob += R_out[i] >= D_out.get(i, 0) - C_initial.get(i, 0), f"Rejection_Outbound_Def_{i}"

# Constraint 4: Maximum Budget
prob += lpSum([C_slot * E[i] for i in N]) <= B_max, "Budget_Constraint"

In [4]:
# We solve the optimization problem

prob.solve()

print("Minimum Total Cost: €", round(value(prob.objective), 2))

# We analyze the results
results = []
for i in N:
    # We only stations present in the initial capacity data
    if i in C_initial and value(S[i]) is not None:
        optimum_capacity = value(S[i])
        initial_capacity = C_initial.get(i)

        results.append({
            'ID_Station': i,
            'Initial Capacity': initial_capacity,
            'Optimum Capacity': optimum_capacity,
            'Expansion Needed': max(0, optimum_capacity - initial_capacity),
            'Rejection Inbound': round(value(R_in[i]), 2),
            'Rejection Outbound': round(value(R_out[i]), 2)
        })

df_results = pd.DataFrame(results)

# We filter stations with expansion or rejections
df_action = df_results[
    (df_results['Expansion Needed'] > 0) | 
    (df_results['Rejection Inbound'] > 0.01) | 
    (df_results['Rejection Outbound'] > 0.01)
]

print("\n--- Top 10 Stations with Optimal Changes or Rejections ---")

print(df_action.sort_values(by='Expansion Needed', ascending=False).head(10).to_string(index=False))
expansion_cost = df_action['Expansion Needed'].sum() * C_slot
print(f"\nTotal Cost of Capacity Expansion: €{expansion_cost:.2f}")

Minimum Total Cost: € 1344030.0

--- Top 10 Stations with Optimal Changes or Rejections ---
 ID_Station  Initial Capacity  Optimum Capacity  Expansion Needed  Rejection Inbound  Rejection Outbound
          1                23              10.0                 0                0.0               723.0
          2                27              10.0                 0                0.0               494.0
          3                19              10.0                 0                0.0              1154.0
          4                27              10.0                 0                0.0               920.0
          5                27              10.0                 0                0.0              1039.0
          6                19              10.0                 0                0.0              1232.0
          7                19              10.0                 0                0.0              1313.0
          8                24              10.0                 0   

### Different Scenario Analysis

#### Why this Analysis?

The initial execution of the linear model resulted in an optimal investment of **€0**, indicating that under the current parameter settings, the Cost of Infrastructure ($C_{slot}$) significantly outweighs the Penalty for Service Failure ($P_{penalty}$). Because this is a linear optimization model, the decision logic is mathematically **binary**: the solver simply calculates whether it is cheaper to build a slot or pay the penalty. To validate the robustness of our expansion plan, we must compare two opposing strategic realities.

* **Scenario A:**
    * **Parameters:** Cost = €500 | Penalty = €5 ($C_{slot} > P_{penalty}$)
    * **Logic:** This represents a financial strategy where budget protection is prioritized over service levels. Since the cost of construction is 100 times higher than the cost of rejecting a user, the mathematical optimum is to **freeze investment** and accept high rejection rates. In this scenario, it is simply "cheaper to pay the fine than to fix the problem."

* **Scenario B:**
    * **Parameters:** Cost = €500 | Penalty = €600 ($P_{penalty} > C_{slot}$)
    * **Logic:** This simulates a strategic shift where BiciMad prioritizes brand reputation. By setting the penalty higher than the construction cost, the model effectively understands that losing a recurring customer is more expensive than building a single slot. This forces the solver to utilize the entire budget to avoid rejections, leading to a massive expansion of the network.

In [6]:
# We compare the two critical realities:
scenarios_A = {
    "A. Financial Focus": {
        "C_slot": 500,       
        "P_rejection": 5      
    },
    "B. Service Focus": {
        "C_slot": 500,       
        "P_rejection": 600 
    }
}

for name, params in scenarios_A.items():
    
    # 1. Initialize Problem
    prob_scen = LpProblem(f"Scenario_{name[:5]}", LpMinimize)
    c_slot = params["C_slot"]
    p_rejection = params["P_rejection"]

    # 2. Variables
    S_var = LpVariable.dicts("Slots_Final", N, lowBound=S_min, upBound=S_max, cat=LpInteger)
    E_var = LpVariable.dicts("Expansion", N, lowBound=0, cat=LpInteger)
    R_in = LpVariable.dicts("R_In", N, lowBound=0, cat=LpContinuous)
    R_out = LpVariable.dicts("R_Out", N, lowBound=0, cat=LpContinuous)
    
    # 3. Objective Function
    prob_scen += (
        lpSum([c_slot * E_var[i] for i in N]) + 
        lpSum([p_rejection * (R_in[i] + R_out[i]) for i in N])
    )
    
    # 4. Constraints
    for i in N:
        # Link Expansion to Final Capacity
        prob_scen += S_var[i] == C_initial.get(i, 0) + E_var[i]
        
        # KEY FIX: Expansion helps BOTH Inbound and Outbound
        # We assume expanding capacity helps satisfy demand in both directions
        prob_scen += R_in[i] >= D_in.get(i, 0) - S_var[i]
        prob_scen += R_out[i] >= D_out.get(i, 0) - S_var[i] 
        
    # Budget Constraint
    prob_scen += lpSum([c_slot * E_var[i] for i in N]) <= B_max

    # 5. Solve
    prob_scen.solve(PULP_CBC_CMD(msg=False))
    
    # 6. Metrics
    nuevos_slots = sum(value(E_var[i]) for i in N)
    inversion = nuevos_slots * c_slot
    
    # Detailed output to debug
    print(f"SCENARIO: {name}")
    print(f"  > Ratio: Cost (€{c_slot}) vs. Penalty (€{p_rejection})")
    print(f"  > Investment Decision:  €{inversion:,.0f}")
    print(f"  > New Slots Built:      {int(nuevos_slots)}")
  
    
    print("-" * 60)

SCENARIO: A. Financial Focus
  > Ratio: Cost (€500) vs. Penalty (€5)
  > Investment Decision:  €0
  > New Slots Built:      0
------------------------------------------------------------
SCENARIO: B. Service Focus
  > Ratio: Cost (€500) vs. Penalty (€600)
  > Investment Decision:  €100,000
  > New Slots Built:      200
------------------------------------------------------------


### Interpretation of Results: 

In Scenario B, the model uses **100% of the available budget (€100,000)**. This behavior is a critical economic indicator of the network's status. The reasons for this total budget usage are the following:

#### 1. Positive Net Benefit
In Scenario B, the parameters create a net benefit for every new slot:
* **Cost to Build:** €500
* **Penalty Saved:** €600
* **Net Benefit per Slot:** +€100

Because the net benefit is positive, the solver determines that **building a slot is always strictly better than not building one**. Consequently, the instruction becomes: *"Build as many slots as physically and financially possible."*


#### 2. The Budget as the Binding Constraint
Since the model is incentivized to expand indefinitely to eliminate penalties, it will continue to add slots until it hits a limit. In this case, the **Budget Constraint ($B_{max}$)** acts as the bottleneck. The model stops investing at exactly €100,000 not because the network is fixed, but simply because it has run out of capital. This indicates that the budget constraint is **binding**.

#### 3. Massive Structural Deficit
The most important strategic insight from this result is the magnitude of the infrastructure gap.
* The fact that the budget is fully exhausted implies that **the cost to fix the network's total deficit is significantly higher than €100,000.**
* If the network only needed €50,000 to solve all rejection problems, the model would have spent €50,000 and stopped.
* Since it hit the €100,000 ceiling immediately, we can conclude that the BiciMad network is **severely underfunded** relative to the actual user demand, assuming a customer-centric service standard.

In [7]:
# We are going to analyze how different budget levels affect the solution
C_SLOT = 500
P_REJECTION = 600 

# 2. Test different Budget levels (from €100k to €5 Million)
budget_levels = [100000, 200000, 500000, 1000000, 2000000, 5000000]

print(f"Analyzing Network Saturation (Fixed Penalty: €{P_REJECTION}) \n")

for b_lev in budget_levels:
    
    # Initialize
    prob = LpProblem(f"Budget_{b_lev}", LpMinimize)
    
    # Variables
    S_var = LpVariable.dicts("Slots", N, lowBound=S_min, upBound=S_max, cat=LpInteger)
    E_var = LpVariable.dicts("Expansion", N, lowBound=0, cat=LpInteger)
    R_in = LpVariable.dicts("R_In", N, lowBound=0, cat=LpContinuous)
    R_out = LpVariable.dicts("R_Out", N, lowBound=0, cat=LpContinuous)
    
    # Objective Function
    prob += (
        lpSum([C_SLOT * E_var[i] for i in N]) + 
        lpSum([P_REJECTION * (R_in[i] + R_out[i]) for i in N])
    )
    
    # Constraints
    for i in N:
        prob += S_var[i] == C_initial.get(i, 0) + E_var[i]
        prob += R_in[i] >= D_in.get(i, 0) - S_var[i]
        prob += R_out[i] >= D_out.get(i, 0) - S_var[i]
        
    # Budget Constraint
    prob += lpSum([C_SLOT * E_var[i] for i in N]) <= b_lev

    # Solve
    prob.solve(PULP_CBC_CMD(msg=False))
    
    # Metrics
    invested = sum(value(E_var[i]) for i in N) * C_SLOT
    nuevos_slots = sum(value(E_var[i]) for i in N)
    
    # Calculate % of Budget Used
    pct_used = (invested / b_lev) * 100
    
    print(f"Budget Available: €{b_lev:,.0f}")
    print(f"  > Amount Spent:   €{invested:,.0f} ({pct_used:.1f}%)")
    print(f"  > Slots Built:    {int(nuevos_slots)}")
    
    if pct_used < 99:
        print(" Network is now fully optimized! ")
    
    print("-" * 60)

Analyzing Network Saturation (Fixed Penalty: €600) 

Budget Available: €100,000
  > Amount Spent:   €100,000 (100.0%)
  > Slots Built:    200
------------------------------------------------------------
Budget Available: €200,000
  > Amount Spent:   €200,000 (100.0%)
  > Slots Built:    400
------------------------------------------------------------
Budget Available: €500,000
  > Amount Spent:   €500,000 (100.0%)
  > Slots Built:    1000
------------------------------------------------------------
Budget Available: €1,000,000
  > Amount Spent:   €1,000,000 (100.0%)
  > Slots Built:    2000
------------------------------------------------------------
Budget Available: €2,000,000
  > Amount Spent:   €2,000,000 (100.0%)
  > Slots Built:    4000
------------------------------------------------------------
Budget Available: €5,000,000
  > Amount Spent:   €3,227,500 (64.5%)
  > Slots Built:    6455
 Network is now fully optimized! 
-----------------------------------------------------------

### Budget Saturation & Structural Deficit Analysis

To determine the true scale of the network's infrastructure deficit, we performed a test by incrementally increasing the available budget ($B_{max}$) under a service-focused strategy (Penalty > Cost). The results reveal the financial requirement to fully optimize the BiciMad network.

#### 1. Complete Budget Usage (€100k – €2M)
For all budget levels up to **2 000 000€**, the model exhibits **100% Budget Utilization**. 
* This consumption indicates massive unfulfilled demand. 
* Even with a 20x increase in the current budget (from 100k€ to 2M€), the network needs more slots, proving that the infrastructure is severely undersized relative to user demand.


#### 2. The Saturation Point (€3,227,500)
The analysis identifies the precise **Saturation Point at 3 227 500€** (6,455 new slots).
* When provided with 5 000 000€, the model stops spending at approx. 3.2M€. 
* **Meaning:** At this investment level, every single station where expansion is physically possible ($S_{max}$) and economically viable has been expanded. Throwing more money at the problem beyond 3.2M€ yields no further benefit because the demand is fully covered.

#### 3. Strategic Conclusion: 
Comparing the saturation point to the current allocation reveals a critical gap:
* **Required Budget:** €3,227,500
* **Current Budget:** €100,000
* **Coverage:** **3.1%**

**Final Insight:** The current budget of €100,000 covers only **~3%** of the network's actual infrastructure needs. To transition from a "Financial Focus" to a "Service Excellence" standard, BiciMad would need to increase its capital allocation by a factor of **32x**. The current investment is not a solution, but merely a patch on a much larger structural problem.


**Transition to Part B:** infrastructure alone does not guarantee service quality. A station with 50 newly built slots is useless if it contains 0 bycicles during a crowded hour, or if it is completely full when someone tries to return a bike.


### Part B: Operational Optimization of Critical Nodes (Non Linear)

Having defined the optimal infrastructure in Part A, we now shift our focus to the **daily operational challenges**. A station's capacity is only effective if it is actively managed to match the fluctuating hourly demand.

In this section, we analyze the inventory dynamics of **Station 57**, selected as a representative critical node due to its high rotation and demand volatility. Unlike the static linear approach used for network expansion, we employ a **Non-Linear Optimization Model** to minimize the Total Daily Cost. This model balances two conflicting objectives:

1.  **Operational Costs:** The linear cost of logistical rebalancing (transporting bikes via trucks).
2.  **Service Quality Costs:** The quadratic penalty associated with service failures (users unable to rent or return bikes).

By moving from a global view to a granular, time-dependent analysis, we aim to define an optimal logistics strategy that ensures availability throughout the day while maintaining economic efficiency.

### Parameters (Input Data)

| Parameter | Description | Source Value |
| :--- | :--- | :--- |
| $H$ | Set of hourly periods (0 to 23). | `parametros_operacionales_bicimad.csv` |
| $\lambda_h$ | Outbound rate of bikes (demand) in period $h$ (bikes/min). | `Flujo_Salida_por_Min` column. |
| $\mu_h$ | Inbound rate of bikes (supply) in period $h$ (bikes/min). | `Flujo_Llegada_por_Min` column. |
| $C_{\text{move}}$ | Cost to rebalance one bicycle for period $h

In [8]:
from scipy.optimize import minimize, LinearConstraint

# We load the data and define the parameters for the optimization model

df_operations = pd.read_csv('parametros_operacionales_bicimad.csv')

N_PERIODS = len(df_operations) # 24 periods (hours)

# Parameters per period (hourly)
LAMBDA_H = df_operations['Flujo_Salida_por_Min'].values 
MU_H = df_operations['Flujo_Llegada_por_Min'].values     
CM = df_operations['Coste_Rebalanceo_Cm'].iloc[0] 
ALPHA = df_operations['Penalizacion_Cuadratica_Alpha'].iloc[0] 

# Assumed parameters
TOTAL_CAPACITY = 24  # Total capacity of bikes in Station 57 
DURATION_H = 60       # Duration of each period in minutes
R_MAX_HOURLY = 10     # Maximum number of bikes to rebalance per hour
INITIAL_STOCK = 12    # Initial stock of bikes at the start of the day

# Total volume per hour (in bikes)
DEMAND_H = LAMBDA_H * DURATION_H  # Bikes demanded per hour
SUPPLY_H = MU_H * DURATION_H       # Bikes supplied per hour

# Initial solution vector: [R_0, ..., R_23, S_0, ..., S_23]
R_initial = np.zeros(N_PERIODS)
S_initial = np.full(N_PERIODS, INITIAL_STOCK) 
X_initial = np.concatenate([R_initial, S_initial])


In [9]:
# Objective Function

def objective_function(X):
    R = X[:N_PERIODS] 
    S = X[N_PERIODS:] 

    # Linear cost: Rebalancing cost
    cost_rebalance = CM * np.sum(np.abs(R))

    # Outbound loss
    loss_outbound = np.maximum(0, DEMAND_H - S)
    # Inbound loss: if bikes arrive but no slots available
    loss_inbound = np.maximum(0, SUPPLY_H + R - TOTAL_CAPACITY)

    # We penalize extreme values to help the optimizer
    cost_nonlinear = ALPHA * (np.sum(loss_outbound**2) + np.sum(loss_inbound**2))
    
    return cost_rebalance + cost_nonlinear

In [10]:
# Definition of Constraints

# Inventory Balance: S_h + R_h + (O_h - D_h) = S_{h+1}
A_eq = np.zeros((N_PERIODS, 2 * N_PERIODS))
b_eq = np.zeros(N_PERIODS)

for h in range(N_PERIODS):
    # S_h - S_{h+1} + R_h = D_h - O_h
    A_eq[h, N_PERIODS + h] = 1 
    A_eq[h, N_PERIODS + (h + 1) % N_PERIODS] = -1 
    A_eq[h, h] = 1 
    
    # Right side: D_h - O_h
    b_eq[h] = DEMAND_H[h] - SUPPLY_H[h]

# Initial Condition: S_0 = INITIAL_STOCK
A_stock_init = np.zeros((1, 2 * N_PERIODS))
A_stock_init[0, N_PERIODS] = 1 
b_stock_init = np.array([INITIAL_STOCK])

A_eq_total = np.vstack([A_eq, A_stock_init])
b_eq_total = np.concatenate([b_eq, b_stock_init])

# Bounds for R_h and S_h
bounds_S = [(0, TOTAL_CAPACITY)] * N_PERIODS 
bounds_R = [(-R_MAX_HOURLY, R_MAX_HOURLY)] * N_PERIODS
bounds_total = bounds_R + bounds_S

linear_constraint = LinearConstraint(A_eq_total, b_eq_total, b_eq_total)

In [11]:
# We solve the optimization problem
result = minimize(
    objective_function,
    X_initial,
    method='SLSQP',
    bounds=bounds_total,
    constraints=[linear_constraint],
    options={'ftol': 1e-6, 'disp': False, 'maxiter': 1000}
)


if result.success:
    X_opt = result.x
    R_opt = X_opt[:N_PERIODS] 
    S_opt = X_opt[N_PERIODS:] 

    df_output = df_operations[['ID_Periodo_h']].copy()
    
    df_output['Outgoing Demand'] = DEMAND_H.round(2)
    df_output['Arrival Supply'] = SUPPLY_H.round(2)
    df_output['Optimal Stock'] = S_opt.round(2)
    df_output['Rebalancing_R_h'] = R_opt.round(2)
    
    # Compute actual rejections based on the solution
    df_output['Arrival Rejection'] = np.maximum(0, df_output['Arrival Supply'] + R_opt - TOTAL_CAPACITY).round(2)
    df_output['Outgoing Rejection'] = np.maximum(0, df_output['Outgoing Demand'] - S_opt).round(2)

    linear_cost = CM * np.sum(np.abs(R_opt)) 
    nonlinear_cost = objective_function(X_opt) - linear_cost
    
    print("\n Results of Optimization for Critical Station ---")

    print(df_output.head(24).to_string(index=False))
    
    print(f"Optimal Total Cost (Daily): €{result.fun:.2f}")
    print(f"  - Linear Rebalancing Cost (Total): €{linear_cost:.2f}")
    print(f"  - Nonlinear Cost (Penalization Total): €{nonlinear_cost:.2f}")



 Results of Optimization for Critical Station ---
 ID_Periodo_h  Outgoing Demand  Arrival Supply  Optimal Stock  Rebalancing_R_h  Arrival Rejection  Outgoing Rejection
            0             7.65            8.03          12.00             0.03                0.0                0.00
            1             5.32            5.29          12.42             0.00                0.0                0.00
            2             4.84            5.39          12.39             0.00                0.0                0.00
            3             2.42            2.48          12.94             0.03                0.0                0.00
            4             1.61            1.68          13.03             0.01                0.0                0.00
            5             1.45            1.52          13.10             0.01                0.0                0.00
            6             1.58            1.61          13.18             0.00                0.0                0.00
     

### Interpretation of Results 

#### 1. Solution Efficiency and Costs
* **Solution Status: `Optimization terminated successfully`**
  The solver (SLSQP) successfully converged to an optimal solution, balancing the cost of moving bicycles with the penalty of rejecting users.
* **Total Daily Cost: `€0.56`**
  * **Linear Rebalancing Cost: `€0.55`**
  * **Non-linear Penalty Cost: `€0.00`**
  The extremely low total cost indicates that the station can be managed almost perfectly. The model found that by spending only **€0.55** on rebalancing, it can completely eliminate user rejections (penalty cost of €0.00).

#### 2. Inventory Stability (Optimal Stock)
* **Stock Management:** The `Stock_Optimo` remains remarkably stable throughout the 24-hour cycle, staying between **11.9** and **13.2** bicycles.
* **Safety Margin:** Since the station capacity is **24 slots**, the model keeps the inventory at approximately **50% capacity**. This is a classic "buffer" strategy: it ensures there are always enough bikes for departures and enough empty docks for arrivals.

#### 3. Rebalancing Strategy ($R_h$)
* **Precision Adjustments:** The model performs very small rebalancing actions (e.g., $+0.03$ or $+0.01$ bikes/min) during most of the day. This suggests that the natural flow of users at Station 57 is relatively balanced.
* **Peak Hour Intervention:** The most significant rebalancing occurs in the evening (Periods 17, 18, and 19):
    * **Period 18:** The model adds bikes ($R_h = 0.60$) to prepare for high evening demand.
    * **Period 19:** The model removes bikes ($R_h = -0.60$) to prevent the station from becoming full as arrivals increase.
* This "just-in-time" rebalancing prevents the station from reaching 0 (empty) or 24 (full).

#### 4. User Experience and Rejections
* **Inbound/Outbound Rejections: `0.00`**
  The primary goal of the model was to minimize the non-linear penalty of rejecting users. The results show **zero rejections** for both people trying to rent a bike and people trying to return one. 
* This demonstrates that with a proactive rebalancing strategy, Station 57 can achieve a **100% service level**.

#### 5. Strategic Conclusion**

**Part B** proves that at the station level, **logistics (rebalancing) is the key to success.**

* **The Power of Rebalancing:** By making tiny, calculated adjustments to the inventory throughout the day, BiciMad can avoid the "empty station" or "full station" syndrome without expensive infrastructure investments.
* **Operational Recommendation:** Focus on "micro-rebalancing" during the transitions between morning and evening peaks to maintain the inventory near the 50% mark.

### Different Scenarios Analysis

We are going to perform an analysis of different scenarios to evaluate the robustness and scalability of this non-linear model. 

* **Scenario A:** it simulates a big event, such as a concert, a football match or a working day end. Everyone wants to take a bike. We are going to measure if our model is able to replace bikes with the needed frequency. We are going to multiply the demand ($\lambda$) by 2.

* **Scenario B:** it simulates a situationship in which everyone wants to park a bike. For example, the morning when everyone is going to work and arrives to the office. It measures the capacity of releasing parking slots. We are multiplying the supplu ($\mu$) by 2.

* **Scenario C:** it simulates a total chaos day. We are multypling $\lambda$ and $\mu$ by 2.

In [12]:
def objective_function_dynamic(X, current_demand, current_supply):
    R = X[:N_PERIODS] 
    S = X[N_PERIODS:] 
    
    # Linear Cost
    cost_rebalance = CM * np.sum(np.abs(R))

    # Non - linear penalties
    loss_outbound = np.maximum(0, current_demand - S)
    loss_inbound = np.maximum(0, current_supply + R - TOTAL_CAPACITY)

    cost_nonlinear = ALPHA * (np.sum(loss_outbound**2) + np.sum(loss_inbound**2))
    
    return cost_rebalance + cost_nonlinear

A_eq = np.zeros((N_PERIODS, 2 * N_PERIODS))
for h in range(N_PERIODS):
    # S_h - S_{h+1} + R_h = D - O
    A_eq[h, N_PERIODS + h] = 1 
    A_eq[h, N_PERIODS + (h + 1) % N_PERIODS] = -1 
    A_eq[h, h] = 1 

# Initial stock condition
A_stock_init = np.zeros((1, 2 * N_PERIODS))
A_stock_init[0, N_PERIODS] = 1 
A_eq_total_local = np.vstack([A_eq, A_stock_init])

# We define different scenarios by modifying LAMBDA_H and MU_H
scenarios = {
    "1. High Demand (Departures x2)": {"lam_factor": 2.0, "mu_factor": 1.0}, 
    "2. High Supply (Arrivals x2)": {"lam_factor": 1.0, "mu_factor": 2.0}, 
    "3. Stress Test (Both x2)": {"lam_factor": 2.0, "mu_factor": 2.0}  
}

print("Starting scenario analysis...")

LAMBDA_ORIGINAL = LAMBDA_H.copy()
MU_ORIGINAL = MU_H.copy()

for name, params in scenarios.items():

    print(f"{'='*60}")
    print(f"Processing Scenario: {name}")

    # We change parameters for each scenario
    LAMBDA_H = LAMBDA_ORIGINAL * params["lam_factor"]
    MU_H = MU_ORIGINAL * params["mu_factor"]
    
    DEMAND_H = LAMBDA_H * DURATION_H 
    SUPPLY_H = MU_H * DURATION_H

    # We update the right-hand side of the equality constraints as demand and supply change
    b_eq_scenario = np.zeros(N_PERIODS)
    for h in range(N_PERIODS):
        b_eq_scenario[h] = DEMAND_H[h] - SUPPLY_H[h]
        
    # We add the initial stock condition
    b_eq_total_scenario = np.concatenate([b_eq_scenario, b_stock_init])
    
    # We create the new constraint for this scenario
    linear_constraint_scenario = LinearConstraint(A_eq_total, b_eq_total_scenario, b_eq_total_scenario)
    
    # We execute the optimization
    result = minimize(
        objective_function_dynamic, 
        X_initial,
        args = (DEMAND_H, SUPPLY_H),
        method='SLSQP',
        bounds=bounds_total,
        constraints=[linear_constraint_scenario], 
        options={'ftol': 1e-6, 'disp': False, 'maxiter': 1000}
    )
    
    # We show the results
    if result.success:
        X_opt = result.x
        R_opt = X_opt[:N_PERIODS] 
        S_opt = X_opt[N_PERIODS:]
        
        # We compute costs and rejections
        linear_cost = CM * np.sum(np.abs(R_opt)) 
        nonlinear_cost = result.fun - linear_cost
        
        # Total rejections (sum over all hours)
        total_rejection_arrival = np.sum(np.maximum(0, SUPPLY_H + R_opt - TOTAL_CAPACITY))
        total_rejection_departure = np.sum(np.maximum(0, DEMAND_H - S_opt))
        
        print(f"  - Total Diary Cost: €{result.fun:.2f}")
        print(f"  - Transport Cost: €{linear_cost:.2f}")
        print(f"  - Penalization: €{nonlinear_cost:.2f}")
        print(f"  - Total Rejections: {total_rejection_arrival + total_rejection_departure:.2f} lost users")

# We reset parameters for next scenario
LAMBDA_H = LAMBDA_ORIGINAL
MU_H = MU_ORIGINAL
DEMAND_H = LAMBDA_H * DURATION_H 
SUPPLY_H = MU_H * DURATION_H

Starting scenario analysis...
Processing Scenario: 1. High Demand (Departures x2)
  - Total Diary Cost: €437.10
  - Transport Cost: €27.13
  - Penalization: €409.98
  - Total Rejections: 17.48 lost users
Processing Scenario: 2. High Supply (Arrivals x2)
  - Total Diary Cost: €27.13
  - Transport Cost: €27.13
  - Penalization: €0.00
  - Total Rejections: 0.00 lost users
Processing Scenario: 3. Stress Test (Both x2)
  - Total Diary Cost: €69.56
  - Transport Cost: €4.54
  - Penalization: €65.02
  - Total Rejections: 5.68 lost users


### Insights from this Analysis:

* The system shows vulnerability in Scenario A, proving that the current logistic capacity is not enough to cover the entire demand in huge events or peak hours. Increasing the fleet capacity is very important in these cases.

* **Robustness in returns:** in contrast, the system handles properly the High Supply case. The station's capacity buffer combined with standard rebalance is capable of absorbing the increase in returns of bikes.

* **Natural Balance:** Scenario C works better than Scenario A. This suggests that the system naturally self balances when the mobility increases globally (more departures and more arrivals). The high volume of incoming bikes helps satisfying the volume of needed bikes for departures, reducing the need of the rebalancing truck.

### Strategic Conclusions:

According to the system's results, we can conclude that logistics should focus on bringing new bikes to the stations rather than removing them. This is because the system struggles the most when the demand increases, leading to a lot of users' discontent.

We have seen that the performance is clearly worse in the scenario where the demand increases.

We are going to evaluate if setting the starting stock as a decision variable helps the station's performance. 

Right now, the initial stock is set as 12, which may not adapt to different scenarios where more bikes are needed for starting the day.

In [13]:
LAMBDA_ORIGINAL = LAMBDA_H.copy()
MU_ORIGINAL = MU_H.copy()

# We define the equality constraint matrix again without the initial stock condition
A_eq = np.zeros((N_PERIODS, 2 * N_PERIODS))
for h in range(N_PERIODS):
    A_eq[h, N_PERIODS + h] = 1 
    A_eq[h, N_PERIODS + (h + 1) % N_PERIODS] = -1  
    A_eq[h, h] = 1 

scenarios = {
    "1. High Demand (Departures x2)":  {"lam_factor": 2.0, "mu_factor": 1.0},
    "2. High Supply (Arrivals x2)": {"lam_factor": 1.0, "mu_factor": 2.0},
    "3. Stress Test (Both x2)":    {"lam_factor": 2.0, "mu_factor": 2.0}
}

for name, params in scenarios.items():
    
    lambda_scenario = LAMBDA_ORIGINAL * params["lam_factor"]
    mu_scenario = MU_ORIGINAL * params["mu_factor"]
    demand_scen = lambda_scenario * DURATION_H 
    supply_scen = mu_scenario * DURATION_H
    
    # Right-hand side of the equation (b_eq)
    b_eq_scenario = np.zeros(N_PERIODS)
    for h in range(N_PERIODS):
        b_eq_scenario[h] = demand_scen[h] - supply_scen[h]
    
    linear_constraint_scenario = LinearConstraint(A_eq, b_eq_scenario, b_eq_scenario)
    
    # We optimize
    result = minimize(
        objective_function_dynamic, 
        X_initial, 
        args=(demand_scen, supply_scen),
        method='SLSQP',
        bounds=bounds_total,
        constraints=[linear_constraint_scenario], 
        options={'ftol': 1e-6, 'disp': False}
    )
    
    if result.success:
        X_opt = result.x
        R_opt = X_opt[:N_PERIODS]
        S_opt = X_opt[N_PERIODS:]
        
        initial_stock = S_opt[0]
        integer_initial_stock = int(round(initial_stock))
        
        linear_cost = CM * np.sum(np.abs(R_opt))
        total_cost = result.fun

        total_rejection_departure = np.sum(np.maximum(0, demand_scen - S_opt))
        total_rejection_arrival = np.sum(np.maximum(0, supply_scen + R_opt - TOTAL_CAPACITY))
        total_rechazos = total_rejection_departure + total_rejection_arrival
        
        print(f"Scenario  {name}")
        print(f"  - Optimal initial stock (rounded): {integer_initial_stock} bikes")
        print(f"  - Total Cost: €{total_cost:.2f}")
        print(f"  - Transport Cost:  €{linear_cost:.2f}")
        print(f"  - Penalization:      €{total_cost - linear_cost:.2f}")
        print(f"  - Users Lost:    {total_rechazos:.2f}")

        # Comparativa rápida si quieres ver diferencia con el modelo anterior
        if name == "1. High Demand (Departures x2)":
            print(f"This used to cost ~437€. The reduced cost is: {437 - total_cost:.2f}€)")
        
        elif name == "3. Stress Test (Both x2)":
            print(f"(This used to cost ~70€. The reduced cost is: {70 - total_cost:.2f}€)")

        print(f"{'='*60}")


# Reset parameters 
LAMBDA_H = LAMBDA_ORIGINAL
MU_H = MU_ORIGINAL

Scenario  1. High Demand (Departures x2)
  - Optimal initial stock (rounded): 23 bikes
  - Total Cost: €382.97
  - Transport Cost:  €27.13
  - Penalization:      €355.85
  - Users Lost:    14.19
This used to cost ~437€. The reduced cost is: 54.03€)
Scenario  2. High Supply (Arrivals x2)
  - Optimal initial stock (rounded): 13 bikes
  - Total Cost: €27.13
  - Transport Cost:  €27.13
  - Penalization:      €0.00
  - Users Lost:    0.00
Scenario  3. Stress Test (Both x2)
  - Optimal initial stock (rounded): 24 bikes
  - Total Cost: €12.51
  - Transport Cost:  €1.62
  - Penalization:      €10.89
  - Users Lost:    2.39
(This used to cost ~70€. The reduced cost is: 57.49€)


We see that by setting the initial stock as a decision variable the system optimizes its performance in the most critical scenarios: the ones in which the demand increases and the costs were higher. 

In both scenarios in which the demand increased, more than 50€ are saved with this approach.